In [7]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
from io import StringIO
# SAS token of the storage account
sas_token = "YOUR_SAS_TOKEN_HERE"
# Storage account details and path to the file
account_name = "YOUR_STORAGE_ACCOUNT_NAME_HERE"
container_name = "wwi"
blob_name = "wwicsvfiles/DimCity.csv"
# Build the URL to the file
blob_url = f"https://{account_name}.blob.core.windows.net/{container_name}/{blob_name}?{sas_token}"
# Create the connection to the storage account
blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=sas_token)
# Create the connection to the CSV file
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
# Download the csv file
blob_data = blob_client.download_blob().content_as_text()
# Read the file into a DataFrame
blob_df = pd.read_csv(StringIO(blob_data))
# Display the DataFrame
display(blob_df)


StatementMeta(, 89b08253-17cb-4693-bbcd-759205341276, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f9d7dbdf-8984-4435-9c50-e3564f8253f8)

In [8]:
# Declare the file name  
file_name = "dim_city"
# Load the DataFrame into a Parquet file on the Lakehouse
blob_df.to_parquet("/lakehouse/default/Files/" + file_name + ".parquet" )


StatementMeta(, 89b08253-17cb-4693-bbcd-759205341276, 10, Finished, Available, Finished)

In [9]:
# Drop the columns 'Valid_From', 'Valid_To', and 'Latitude'
blob_df.drop(['Valid_From', 'Valid_To', 'Latitude'], axis=1, inplace=True)
# Create a new column by concatenating 'City', 'State Province', and 'Country'
blob_df['Location'] = blob_df['City'] + ', ' + blob_df['State_Province'] + ', ' + blob_df['Country']
# Define the destination table name
table_name = "dim_city"
# Convert the Pandas dataframe to a Spark dataframe
spark_df = spark.createDataFrame(blob_df)
# Create the delta table in the Lakehouse
spark_df.write.mode("overwrite").format("delta").saveAsTable(table_name)

StatementMeta(, 89b08253-17cb-4693-bbcd-759205341276, 11, Finished, Available, Finished)

In [10]:
# Enabling V-Order
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
# Enabling optimized writing for Delta files
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
# Defining the destination table name
table_name = "dim_city_optimized"
# Creating the delta table in the Lakehouse
spark_df.write.format("delta").mode("append").saveAsTable(table_name)


StatementMeta(, 89b08253-17cb-4693-bbcd-759205341276, 12, Finished, Available, Finished)

In [11]:
df = spark.sql("SELECT * FROM Customer_analysis.dim_city_optimized LIMIT 1000")
display(df)

StatementMeta(, 89b08253-17cb-4693-bbcd-759205341276, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d27f5ec8-708d-408a-b231-e4019d721b25)